In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow import feature_column

In [7]:
df = pd.read_csv('7y_nona.csv')
df.columns

Index(['death', 'recurrence', 'metastasis', 'progression', 'age', 'gender',
       'treatment', 'IC.N', 'CC.N', 'AC.N', 'pathology', 'T', 'N',
       'UICC.Stage', 'smoking', 'family.history', 'EBVDNA', 'BMI', 'WBC',
       'NEU', 'LYM', 'MON', 'HGB', 'PLT', 'LDH', 'ALB', 'TBIL', 'DBIL', 'IBIL',
       'BUN', 'CRE', 'UA', 'CHO', 'TG', 'HDL.C', 'LDL.C', 'ApoA1', 'ApoB',
       'CRP', 'VCA.IgA', 'EA.IgA'],
      dtype='object')

然后我们来分割数据。这里使用的是 Scikit-learn 中的 train_test_split 函数。指定分割比例即可。
我们先按照 80:20 的比例，把总体数据分成训练集和测试集。

In [9]:
tf.random.set_seed(1)
train, test = train_test_split(df, test_size=0.2, random_state=1)

然后，再把现有训练集的数据，按照 80:20 的比例，分成最终的训练集，以及验证集。

In [10]:
train,valid = train_test_split(train, test_size=0.25, random_state=1)

这里，我们都指定了 random_state ，为的是保证咱们随机分割的结果一致。
我们看看几个不同集合的长度。

In [11]:
print(len(train))
print(len(valid))
print(len(test))

486
162
163


In [22]:
feature_columns = []
numeric_columns = ['age', 'EBVDNA', 'BMI', 'WBC',
       'NEU', 'LYM', 'MON', 'HGB', 'PLT', 'LDH', 'ALB', 'TBIL', 'DBIL', 'IBIL',
       'BUN', 'CRE', 'UA', 'CHO', 'TG', 'HDL-C', 'LDL-C', 'ApoA1', 'ApoB',
       'CRP', 'VCA-IgA', 'EA-IgA']
for header in numeric_columns:
    feature_columns.append(feature_column.numeric_column(header))
    
    
categorical_columns = ['gender',
       'treatment', 'IC.N', 'CC.N', 'AC.N', 'pathology', 'T', 'N',
       'UICC.Stage', 'smoking', 'family.history',]

def get_one_hot_from_categorical(colname):
    categorical = feature_column.categorical_column_with_vocabulary_list(colname, train[colname].unique().tolist())
    return feature_column.indicator_column(categorical)



for col in categorical_columns:
    feature_columns.append(get_one_hot_from_categorical(col))

feature_columns

[NumericColumn(key='age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='EBVDNA', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='BMI', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='WBC', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='NEU', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='LYM', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='MON', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='HGB', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='PLT', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='LDH', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='ALB', s

In [23]:
from tensorflow.keras import layers
feature_layer = layers.DenseFeatures(feature_columns)
feature_layer

In [26]:
model = keras.Sequential([
    feature_layer,
    layers.Dense(200, activation='relu'),
    layers.Dense(100, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

def df_to_tfdata(df, shuffle=True, bs=32):
    df = df.copy()
    labels = df.pop('progression')
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(df), seed=1)
        ds = ds.batch(bs)
        return ds
    
train_ds = df_to_tfdata(train)
valid_ds = df_to_tfdata(valid, shuffle=False)
test_ds = df_to_tfdata(test, shuffle=False)

ValueError: Attempt to convert a value (0) with an unsupported type (<class 'numpy.int64'>) to a Tensor.

In [18]:
model.fit(train_ds,
          validation_data=valid_ds,
          epochs=5)


NameError: name 'train_ds' is not defined